In [1]:
!pip install datasets
!pip install textstat
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.3 MB/s eta 0:00:00
   ━━━

In [2]:
!gdown --id 1_N7Y1Zx63w1KorOiwHTpr4PVaSwjndsl
!unzip -qq -u learning-agency-lab-automated-essay-scoring-2.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_N7Y1Zx63w1KorOiwHTpr4PVaSwjndsl
To: /content/learning-agency-lab-automated-essay-scoring-2.zip
100% 12.5M/12.5M [00:00<00:00, 54.1MB/s]


In [3]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    normalizers,
    processors,
    trainers,
    Tokenizer
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

import gc

import spacy
from collections import Counter

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import textstat
from spellchecker import SpellChecker

from sentence_transformers import SentenceTransformer, models
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
import torch

import copy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import Dataset
from glob import glob
import re
import random
import warnings
warnings.filterwarnings('ignore')
import gc
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import re
import random
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.metrics import cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
from sklearn.linear_model import SGDClassifier
import pickle
from scipy.special import softmax
import kagglehub

nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('vader_lexicon')

tqdm.pandas()

nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [4]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

## Textstat Features

In [5]:
def textstat_features(text):
    features = {}
    features['flesch_reading_ease'] = textstat.flesch_reading_ease(text)
    features['flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(text)
    features['smog_index'] = textstat.smog_index(text)
    features['coleman_liau_index'] = textstat.coleman_liau_index(text)
    features['automated_readability_index'] = textstat.automated_readability_index(text)
    features['dale_chall_readability_score'] = textstat.dale_chall_readability_score(text)
    features['difficult_words'] = textstat.difficult_words(text)
    features['linsear_write_formula'] = textstat.linsear_write_formula(text)
    features['gunning_fog'] = textstat.gunning_fog(text)
    features['text_standard'] = textstat.text_standard(text, float_output=True)
    features['spache_readability'] = textstat.spache_readability(text)
    features['mcalpine_eflaw'] = textstat.mcalpine_eflaw(text)
    features['reading_time'] = textstat.reading_time(text)
    features['syllable_count'] = textstat.syllable_count(text)
    features['lexicon_count'] = textstat.lexicon_count(text)
    features['monosyllabcount'] = textstat.monosyllabcount(text)

    return features

train['textstat_features'] = train['full_text'].apply(textstat_features)
train_textstat = pd.DataFrame(train['textstat_features'].tolist())

test['textstat_features'] = test['full_text'].apply(textstat_features)
test_textstat = pd.DataFrame(test['textstat_features'].tolist())

train_textstat.head()

,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,spache_readability,mcalpine_eflaw,reading_time,syllable_count,lexicon_count,monosyllabcount
0,57.98,14.7,11.7,8.19,18.3,8.74,60,13.000000,17.33,9.0,7.28,54.5,31.97,634,498,404
1,87.55,5.4,6.8,4.99,6.2,6.31,24,6.714286,7.48,7.0,3.92,25.7,19.60,398,332,275
2,65.15,9.9,11.5,8.94,11.6,7.24,67,15.500000,11.49,12.0,5.12,32.6,36.96,767,550,417
3,58.32,10.4,13.2,10.97,12.9,8.50,78,15.750000,11.91,11.0,5.34,29.6,33.01,685,448,291
4,54.66,11.8,13.0,10.57,13.9,7.79,55,19.666667,12.64,13.0,5.61,35.7,26.71,562,373,241


## Linguistic Features

In [6]:
def extract_linguistic_features(text):

    doc = nlp(text)
    features = {}

    # NER Features
    entity_counts = {"GPE": 0, "PERCENT": 0, "NORP": 0, "ORG": 0, "CARDINAL": 0, "MONEY": 0, "DATE": 0,
                    "LOC": 0, "PERSON": 0, "QUANTITY": 0, "EVENT": 0, "ORDINAL": 0, "WORK_OF_ART": 0,
                    "LAW": 0, "PRODUCT": 0, "TIME": 0, "FAC": 0, "LANGUAGE": 0}
    for entity in doc.ents:
        if entity.label_ in entity_counts:
            entity_counts[entity.label_] += 1
    features['NER_Features'] = entity_counts

    # POS Features
    pos_counts = {"ADJ": 0, "NOUN": 0, "VERB": 0, "SCONJ": 0, "PRON": 0, "PUNCT": 0, "DET": 0, "AUX": 0,
                "PART": 0, "ADP": 0, "SPACE": 0, "CCONJ": 0, "PROPN": 0, "NUM": 0, "ADV": 0,
                "SYM": 0, "INTJ": 0, "X": 0}
    for token in doc:
        if token.pos_ in pos_counts:
            pos_counts[token.pos_] += 1
    features['POS_Features'] = pos_counts

    # tag Features
    tags = {"RB": 0, "-RRB-": 0, "PRP$": 0, "JJ": 0, "TO": 0, "VBP": 0, "JJS": 0, "DT": 0, "''": 0, "UH": 0, "RBS": 0, "WRB": 0, ".": 0,
        "HYPH": 0, "XX": 0, "``": 0, "SYM": 0, "VB": 0, "VBN": 0, "WP": 0, "CC": 0, "LS": 0, "POS": 0, "NN": 0, ",": 0, "NNPS": 0,
          "RP": 0, ":": 0, "$": 0, "PDT": 0, "VBZ": 0, "VBD": 0, "JJR": 0, "-LRB-": 0, "IN": 0, "RBR": 0, "WDT": 0, "EX": 0, "MD": 0,
            "_SP": 0, "NNP": 0, "CD": 0, "VBG": 0, "NNS": 0, "PRP": 0}

    for token in doc:
        if token.tag_ in tags:
            tags[token.tag_] += 1
    features['tag_Features'] = tags

    # tense features
    tenses = [i.morph.get("Tense") for i in doc]
    tenses = [i[0] for i in tenses if i]
    tense_counts = Counter(tenses)
    features['past_tense_ratio'] = tense_counts.get("Past", 0) / (tense_counts.get("Pres", 0) + tense_counts.get("Past", 0) + 1e-5)
    features['present_tense_ratio'] = tense_counts.get("Pres", 0) / (tense_counts.get("Pres", 0) + tense_counts.get("Past", 0) + 1e-5)


    # len features

    features['word_count'] = len(doc)
    features['sentence_count'] = len([sentence for sentence in doc.sents])
    features['words_per_sentence'] = features['word_count'] / features['sentence_count']
    features['std_words_per_sentence'] = np.std([len(sentence) for sentence in doc.sents])

    features['unique_words'] = len(set([token.text for token in doc]))
    features['lexical_diversity'] = features['unique_words'] / features['word_count']

    paragraph = text.split('\n\n')

    features['paragraph_count'] = len(paragraph)

    features['avg_chars_by_paragraph'] = np.mean([len(paragraph) for paragraph in paragraph])
    features['avg_words_by_paragraph'] = np.mean([len(nltk.word_tokenize(paragraph)) for paragraph in paragraph])
    features['avg_sentences_by_paragraph'] = np.mean([len(nltk.sent_tokenize(paragraph)) for paragraph in paragraph])

    # sentiment features
    analyzer = SentimentIntensityAnalyzer()
    sentences = nltk.sent_tokenize(text)

    compound_scores, negative_scores, positive_scores, neutral_scores = [], [], [], []
    for sentence in sentences:
        scores = analyzer.polarity_scores(sentence)
        compound_scores.append(scores['compound'])
        negative_scores.append(scores['neg'])
        positive_scores.append(scores['pos'])
        neutral_scores.append(scores['neu'])

    features["mean_compound"] = np.mean(compound_scores)
    features["mean_negative"] = np.mean(negative_scores)
    features["mean_positive"] = np.mean(positive_scores)
    features["mean_neutral"] = np.mean(neutral_scores)

    features["std_compound"] = np.std(compound_scores)
    features["std_negative"] = np.std(negative_scores)
    features["std_positive"] = np.std(positive_scores)
    features["std_neutral"] = np.std(neutral_scores)

    return features

train['linguistic_features'] = train['full_text'].progress_apply(extract_linguistic_features)

train_linguistic = pd.json_normalize(train['linguistic_features'])



test['linguistic_features'] = test['full_text'].progress_apply(extract_linguistic_features)

test_linguistic = pd.json_normalize(test['linguistic_features'])

train_linguistic.head()

  0%|          | 0/17307 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,past_tense_ratio,present_tense_ratio,word_count,sentence_count,words_per_sentence,std_words_per_sentence,unique_words,lexical_diversity,paragraph_count,avg_chars_by_paragraph,...,tag_Features.RBR,tag_Features.WDT,tag_Features.EX,tag_Features.MD,tag_Features._SP,tag_Features.NNP,tag_Features.CD,tag_Features.VBG,tag_Features.NNS,tag_Features.PRP
0,0.275362,0.724638,552,13,42.461538,34.078225,248,0.449275,1,2677.000000,...,1,4,2,10,6,26,12,6,35,29
1,0.160714,0.839286,377,20,18.850000,11.127781,169,0.448276,5,332.200000,...,0,6,5,10,4,10,2,10,15,28
2,0.158730,0.841270,611,25,24.440000,8.168623,246,0.402619,4,767.750000,...,2,9,4,19,4,0,2,11,39,20
3,0.090909,0.909091,516,21,24.571429,10.135141,242,0.468992,5,538.600000,...,1,2,2,11,4,20,6,14,27,19
4,0.183673,0.816326,428,16,26.750000,18.122845,159,0.371495,6,366.333333,...,3,3,0,3,7,31,4,10,13,10


In [7]:
tag_cols = [col for col in train_linguistic.columns if col.startswith('tag')]
col_cols = [col for col in train_linguistic.columns if col.startswith('col')]
pos_cols = [col for col in train_linguistic.columns if col.startswith('pos')]

for col in tag_cols:
    train_linguistic[f"{col}_ratio"] = train_linguistic[col] / train_linguistic['word_count']
    test_linguistic[f"{col}_ratio"] = test_linguistic[col] / test_linguistic['word_count']

for col in col_cols:
    test_linguistic[f"{col}_ratio"] = test_linguistic[col] / test_linguistic['word_count']

for col in pos_cols:
    test_linguistic[f"{col}_ratio"] = test_linguistic[col] / test_linguistic['word_count']

train_linguistic.head()

,past_tense_ratio,present_tense_ratio,word_count,sentence_count,words_per_sentence,std_words_per_sentence,unique_words,lexical_diversity,paragraph_count,avg_chars_by_paragraph,...,tag_Features.RBR_ratio,tag_Features.WDT_ratio,tag_Features.EX_ratio,tag_Features.MD_ratio,tag_Features._SP_ratio,tag_Features.NNP_ratio,tag_Features.CD_ratio,tag_Features.VBG_ratio,tag_Features.NNS_ratio,tag_Features.PRP_ratio
0,0.275362,0.724638,552,13,42.461538,34.078225,248,0.449275,1,2677.000000,...,0.001812,0.007246,0.003623,0.018116,0.010870,0.047101,0.021739,0.010870,0.063406,0.052536
1,0.160714,0.839286,377,20,18.850000,11.127781,169,0.448276,5,332.200000,...,0.000000,0.015915,0.013263,0.026525,0.010610,0.026525,0.005305,0.026525,0.039788,0.074271
2,0.158730,0.841270,611,25,24.440000,8.168623,246,0.402619,4,767.750000,...,0.003273,0.014730,0.006547,0.031097,0.006547,0.000000,0.003273,0.018003,0.063830,0.032733
3,0.090909,0.909091,516,21,24.571429,10.135141,242,0.468992,5,538.600000,...,0.001938,0.003876,0.003876,0.021318,0.007752,0.038760,0.011628,0.027132,0.052326,0.036822
4,0.183673,0.816326,428,16,26.750000,18.122845,159,0.371495,6,366.333333,...,0.007009,0.007009,0.000000,0.007009,0.016355,0.072430,0.009346,0.023364,0.030374,0.023364


In [8]:
merged_df = pd.concat([train_textstat, train_linguistic], axis=1)

merged_df_test = pd.concat([test_textstat, test_linguistic], axis=1)

## Error Counts

In [9]:
spell = SpellChecker()

def spell_check(text):
    words = nltk.word_tokenize(text)
    misspelled = spell.unknown(words)

    mispelled_count = len(misspelled)
    misspelled_ratio = mispelled_count / len(words)

    return mispelled_count, misspelled_ratio

train['spell_check_features'] = train['full_text'].progress_apply(spell_check)

spell_check_df = pd.DataFrame(train['spell_check_features'].tolist(), columns=['misspelled_count', 'misspelled_ratio'])

test['spell_check_features'] = test['full_text'].progress_apply(spell_check)

test_check_df = pd.DataFrame(test['spell_check_features'].tolist(), columns=['misspelled_count', 'misspelled_ratio'])

spell_check_df.head()

  0%|          | 0/17307 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,misspelled_count,misspelled_ratio
0,30,0.055046
1,13,0.035040
2,12,0.019835
3,15,0.029354
4,15,0.035885


In [10]:
merged_df = pd.concat((merged_df, spell_check_df), axis=1)

merged_df_test = pd.concat((merged_df_test, test_check_df), axis=1)

# Feedback Features

In [5]:
feedback_path = kagglehub.dataset_download("datafan07/feedback-data")
sentpath = kagglehub.dataset_download("datafan07/sent-debsmall")
feedback_df = pd.read_csv(feedback_path+'/feedback_data.csv')

feed_embeds = []

merged_embeds = []

test_embeds = []

for i in range(5):
    model_path = sentpath + f'/deberta_small_trained/temp_fold{i}_checkpoints'
    word_embedding_model = models.Transformer(model_path, max_seq_length=1024)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    model.half()
    model = model.to('cuda')

    feed_custom_embeddings_train = model.encode(feedback_df.loc[:, 'full_text'].values, device='cuda',
                                                show_progress_bar=True, normalize_embeddings=True)

    feed_embeds.append(feed_custom_embeddings_train)

    merged_custom_embeddings = model.encode(train.loc[:, 'full_text'].values, device='cuda',
                                            show_progress_bar=True, normalize_embeddings=True)

    merged_embeds.append(merged_custom_embeddings)


    test_custom_embeddings = model.encode(test.loc[:, 'full_text'].values, device='cuda',
                                            show_progress_bar=True, normalize_embeddings=True)

    test_embeds.append(test_custom_embeddings)

feed_embeds = np.mean(feed_embeds, axis=0)
merged_embeds = np.mean(merged_embeds, axis=0)
test_embeds = np.mean(test_embeds, axis=0)

100%|██████████| 2.80M/2.80M [00:01<00:00, 2.43MB/s]

Extracting files...


100%|██████████| 2.04G/2.04G [01:49<00:00, 20.0MB/s]

Extracting files...


Batches:   0%|          | 0/123 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/541 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/123 [00:00<?, ?it/s]

Batches:   0%|          | 0/541 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
targets = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']


ridge = Ridge(alpha=1.0)

multioutputregressor = MultiOutputRegressor(ridge)



multioutputregressor.fit(feed_embeds, feedback_df.loc[:, targets])

MultiOutputRegressor(estimator=Ridge())

In [7]:
feedback_predictions = multioutputregressor.predict(merged_embeds)

feedback_predictions_df = pd.DataFrame(feedback_predictions, columns=targets)

test_feedback_predictions = multioutputregressor.predict(test_embeds)

test_feedback_predictions_df = pd.DataFrame(test_feedback_predictions, columns=targets)

feedback_predictions_df.head()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,2.768141,2.728850,3.093324,2.903787,2.816990,2.510284
1,3.330804,3.268910,3.471334,3.344031,3.400761,3.189667
2,4.136734,4.146534,4.322049,4.295417,4.234236,4.076422
3,3.830385,3.729888,4.011009,3.816862,3.779596,3.744656
4,3.383251,3.411404,3.551325,3.412763,3.537275,3.339451


In [8]:
feedback_predictions_df.to_csv('./feedback_train.csv',index=False)
test_feedback_predictions_df.to_csv('./feedback_test.csv',index=False)

In [11]:
feedback_predictions_df = pd.read_csv('./feedback_train.csv')
test_feedback_predictions_df = pd.read_csv('./feedback_test.csv')

In [12]:
merged_df = pd.concat((merged_df, feedback_predictions_df), axis=1)

merged_df_test = pd.concat((merged_df_test, test_feedback_predictions_df), axis=1)

#paragraph, sentence, words

In [13]:
columns = [
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]

# 载入训练集和测试集，同时对full_text数据使用\n\n字符分割为列表，重命名为paragraph
# Load training and testing sets, while using \ n \ n character segmentation to list and renaming to paragraph for full_text data
train_df = pl.read_csv("train.csv").with_columns(columns)
test_df = pl.read_csv("test.csv").with_columns(columns)

In [14]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    x = x.lower()
    x = removeHTML(x)
    x = re.sub("@\w+", '',x)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub("http\w+", '',x)
    x = re.sub(r"\s+", " ", x)
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = x.strip()
    return x

In [15]:
# 段落特征
# paragraph features
def Paragraph_Preprocess(tmp):
    # 将段落列表扩展为一行行的数据
    # Expand the paragraph list into several lines of data
    tmp = tmp.explode('paragraph')
    # 段落预处理
    # Paragraph preprocessing
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    # 计算每一个段落的长度
    # Calculate the length of each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    # 计算每一个段落中句子的数量和单词的数量
    # Calculate the number of sentences and words in each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp
# feature_eng
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
def Paragraph_Eng(train_tmp):
    aggs = [
        # 统计段落长度大于和小于 i 值的个数
        # Count the number of paragraph lengths greater than and less than the i-value
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_{i}_cnt") for i in [50,75,100,125,150,175,200,250,300,350,400,500,600,700] ],
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_{i}_cnt") for i in [25,49]],
        # 其他
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea],
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df
tmp = Paragraph_Preprocess(train_df)
train_feats = Paragraph_Eng(tmp)
train_feats['score'] = train_df['score']
# 获取特征名称
# Obtain feature names
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))

tmp = Paragraph_Preprocess(test_df)
test_feats = Paragraph_Eng(tmp)

Features Number:  31


In [16]:
# sentence feature
def Sentence_Preprocess(tmp):
    # 对full_text预处理，并且使用句号分割出文本的句子
    # Preprocess full_text and use periods to segment sentences in the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=".").alias("sentence"))
    tmp = tmp.explode('sentence')
    # 计算句子的长度
    # Calculate the length of a sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    # 筛选出句子长度大于15的那一部分数据
    # Filter out the portion of data with a sentence length greater than 15
    tmp = tmp.filter(pl.col('sentence_len')>=15)
    # 统计每一句中单词的数量
    # Count the number of words in each sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))

    return tmp
# feature_eng
sentence_fea = ['sentence_len','sentence_word_cnt']
def Sentence_Eng(train_tmp):
    aggs = [
        # 统计句子长度大于 i 的句子个数
        # Count the number of sentences with a length greater than i
        *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_{i}_cnt") for i in [15,50,100,150,200,250,300] ],
        # 其他
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in sentence_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in sentence_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in sentence_fea],
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Sentence_Preprocess(train_df)
# 将新生成的特征数据和之前生成的特征数据合并
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))

tmp = Sentence_Preprocess(test_df)
test_feats = test_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

Features Number:  48


In [17]:
# word feature
def Word_Preprocess(tmp):
    # 对full_text预处理，并且使用空格符分割出文本的单词
    # Preprocess full_text and use spaces to separate words from the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=" ").alias("word"))
    tmp = tmp.explode('word')
    # 计算每一个的单词长度
    # Calculate the length of each word
    tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))
    # 删除单词长度为0的数据
    # Delete data with a word length of 0
    tmp = tmp.filter(pl.col('word_len')!=0)

    return tmp
# feature_eng
def Word_Eng(train_tmp):
    aggs = [
        # 统计单词长度大于 i+1 的单词个数
        # Count the number of words with a length greater than i+1
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_{i+1}_cnt") for i in range(15) ],
        # 其他
        # other
        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        pl.col('word_len').quantile(0.25).alias(f"word_len_q1"),
        pl.col('word_len').quantile(0.50).alias(f"word_len_q2"),
        pl.col('word_len').quantile(0.75).alias(f"word_len_q3"),
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Word_Preprocess(train_df)
# 将新生成的特征数据和之前生成的特征数据合并
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))

tmp = Word_Preprocess(test_df)
test_feats = test_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

Features Number:  69


#TfidfVectorizer, CountVectorizer

In [ ]:
# TfidfVectorizer parameter
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(4,8),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)
# 将全部数据集都填充进TfidfVectorizer里，这可能会造成泄露和过于乐观的CV分数
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
train_tfid = vectorizer.fit_transform([i for i in train_df['full_text']])
# 转换为数组
# Convert to array
dense_matrix = train_tfid.toarray()
# 转换为dataframe
# Convert to dataframe
df = pd.DataFrame(dense_matrix)
# 重命名特征
# rename features
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
# 将新生成的特征数据和之前生成的特征数据合并
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

test_tfid = vectorizer.transform([i for i in test_df['full_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df, on='essay_id', how='left')

In [ ]:
vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(3,5),
            min_df=0.10,
            max_df=0.85,
)
train_tfid = vectorizer_cnt.fit_transform([i for i in train_df['full_text']])
dense_matrix = train_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
train_feats = train_feats.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

test_tfid = vectorizer_cnt.transform([i for i in test_df['full_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df, on='essay_id', how='left')

#Pretrain_model score

In [18]:
import joblib

modelpath = kagglehub.dataset_download("hideyukizushi/aes2-400-20240419134941")
deberta_oof = joblib.load(modelpath+'/oof.pkl')
print(deberta_oof.shape, train_feats.shape)

for i in range(6):
    train_feats[f'deberta_oof_{i}'] = deberta_oof[:, i]

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))

train_feats.shape

100%|██████████| 18.5G/18.5G [04:57<00:00, 66.8MB/s]

Extracting files...


(17307, 6) (17307, 71)
Features Number:  75


(17307, 77)

In [19]:
MAX_LENGTH = 1024
MODEL_PATH = modelpath + '/*/*'
EVAL_BATCH_SIZE = 1
models = glob(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(models[0])

def tokenize(sample):
    return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)

ds = Dataset.from_pandas(test).map(tokenize).remove_columns(['essay_id', 'full_text'])

args = TrainingArguments(
    ".",
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    report_to="none"
)

predictions = []
for model in models:
    model = AutoModelForSequenceClassification.from_pretrained(model)
    trainer = Trainer(
        model=model,
        args=args,
        data_collator=DataCollatorWithPadding(tokenizer),
        tokenizer=tokenizer
    )
    preds = trainer.predict(ds).predictions
    predictions.append(softmax(preds, axis=-1))
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()

predicted_score = 0.
for p in predictions:
    predicted_score += p

predicted_score /= len(predictions)

# Meta Features(deberta-v3-large)
for i in range(6):
    test_feats[f'deberta_oof_{i}'] = predicted_score[:, i]

# Features number
feature_names = list(filter(lambda x: x not in ['essay_id','score'], test_feats.columns))
print('Features number: ',len(feature_names))

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Features number:  75


In [20]:
merged_df = pd.concat((merged_df, train_feats), axis=1)

merged_df_test = pd.concat((merged_df_test, test_feats), axis=1)

In [21]:
merged_df = merged_df.drop(['essay_id','score'], axis=1)
merged_df_test = merged_df_test.drop(['essay_id'], axis=1)

#Predict

In [22]:
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = y_true + a
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True


# metric and objective based on public notebooks

def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess
a = 2.998
b = 1.092



skf = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)

scores = []

train['oof'] = 0

test_preds = []

for fold, (train_idx, valid_idx) in enumerate(skf.split(train['full_text'], train['score'])):
    print(f"Fold: {fold}")
    print(f"Train size: {len(train_idx)}")
    print(f"Valid size: {len(valid_idx)}")
    print()


    X_train = merged_df.iloc[train_idx].values
    X_valid = merged_df.iloc[valid_idx].values


    y_train = train['score'].astype('float32').values[train_idx]
    y_valid = train['score'].astype('float32').values[valid_idx]


    y_train = y_train -a
    y_valid = y_valid -a



    model = lgb.LGBMRegressor(
                objective = qwk_obj,
                metrics = 'None',
                learning_rate = 0.01,
                n_estimators=10000,
                random_state=42,
                extra_trees=True,
                class_weight='balanced',
                verbosity = - 1)

    callbacks = [lgb.early_stopping(500, verbose=True, first_metric_only=True), lgb.log_evaluation(period=500)]


    predictor = model.fit(X_train,
                                  y_train,
                                  eval_names=['train', 'valid'],
                                  eval_set=[(X_train, y_train), (X_valid, y_valid)],
                                  eval_metric=quadratic_weighted_kappa,
                                  callbacks=callbacks,)

    valid_preds = predictor.predict(X_valid)

    train.loc[valid_idx, 'oof'] = valid_preds + a

    score = quadratic_weighted_kappa(y_valid, valid_preds)
    scores.append(score[1])

    test_preds.append(predictor.predict(merged_df_test) + a)

    print(f"Train QWK: {score}")

print(f"Mean QWK: {np.mean(scores)}")

Fold: 0
Train size: 16153
Valid size: 1154

Training until validation scores don't improve for 500 rounds
[500]	train's QWK: 0.862371	valid's QWK: 0.848836
[1000]	train's QWK: 0.87434	valid's QWK: 0.852189
[1500]	train's QWK: 0.886101	valid's QWK: 0.852806
Early stopping, best iteration is:
[1170]	train's QWK: 0.878462	valid's QWK: 0.853893
Evaluated only: QWK
Train QWK: ('QWK', 0.8538931575235835, True)
Fold: 1
Train size: 16153
Valid size: 1154

Training until validation scores don't improve for 500 rounds
[500]	train's QWK: 0.862435	valid's QWK: 0.852685
[1000]	train's QWK: 0.874032	valid's QWK: 0.853366
Early stopping, best iteration is:
[816]	train's QWK: 0.870019	valid's QWK: 0.854121
Evaluated only: QWK
Train QWK: ('QWK', 0.8541205262785312, True)
Fold: 2
Train size: 16153
Valid size: 1154

Training until validation scores don't improve for 500 rounds
[500]	train's QWK: 0.862284	valid's QWK: 0.842787
[1000]	train's QWK: 0.874423	valid's QWK: 0.841145
Early stopping, best iterati

In [23]:
def classify_with_thresholds(oof_clip, optimal_thresholds):

    # Класифікуємо значення oof за допомогою оптимальних порогів
    classified_oof = np.empty_like(oof_clip)
    for i, threshold in enumerate(optimal_thresholds):
        if i == 0:
            classified_oof[oof_clip < threshold] = 1

        else:
            classified_oof[(oof_clip >= optimal_thresholds[i-1]) & (oof_clip < threshold)] = i + 1

        if i == 4:
            classified_oof[oof_clip >= threshold] = 6

    return classified_oof.astype(int)

In [25]:
optimal_thresholds = [1.5, 2.5, 3.5, 4.5, 5.5]
oof_clip = train['oof'].clip(1, 6)
classified_oof =classify_with_thresholds(oof_clip, optimal_thresholds)
best_kappa_score = cohen_kappa_score(train['score'].astype(int).values, classified_oof, weights='quadratic')

for i in range(5):
    i_threshold=optimal_thresholds[i]-0.5
    for j in range(1000):
        thresholds=[x for x in optimal_thresholds]
        thresholds[i]=i_threshold+j/1000

        classified_oof = classify_with_thresholds(oof_clip, thresholds)
        threshold_kappa_score=cohen_kappa_score(train['score'].astype(int).values, classified_oof, weights='quadratic')

        if threshold_kappa_score>=best_kappa_score:
            best_kappa_score=threshold_kappa_score
            optimal_thresholds[i]=thresholds[i]

optimal_thresholds

[1.534, 2.5, 3.459, 4.396, 5.302]

In [26]:
final_preds = (np.mean(test_preds, axis=0))
light_gbm_preds = np.clip(final_preds, 1, 6)
light_gbm_preds = classify_with_thresholds(light_gbm_preds, optimal_thresholds)

In [27]:
sample_submission['score'] = light_gbm_preds
sample_submission.to_csv('submission.csv', index=False)